# Trabalho Final - Aprendizado por Reforço

https://www.youtube.com/watch?app=desktop&v=bD6V3rcr_54&t=0s

https://www.jmlr.org/papers/v22/20-1364.html


https://gymnasium.farama.org/introduction/create_custom_env/

https://github.com/github0apurva/TabularData_Reinforcement

https://github.com/github0apurva/TabularData_Reinforcement/tree/main

https://stackoverflow.com/questions/58964267/how-to-create-an-openai-gym-observation-space-with-multiple-features

https://docs.ray.io/en/master/rllib/index.html

https://gymnasium.farama.org/api/spaces/utils/#gymnasium.spaces.utils.flatten_space

https://github.com/udacity/deep-reinforcement-learning/blob/master/p3_collab-compet/Soccer.ipynb

https://github.com/bryanoliveira/soccer-twos-env

https://github.com/google-research/football


**https://medium.com/@tom.kaminski01/reinforcement-learning-for-f9a28632914f**

**https://gymnasium.farama.org/introduction/create_custom_env/**

In [2]:
import numpy as np
import pandas as pd
import random
from gym import Env
from gym.spaces import Discrete, Box, Dict
from gym.wrappers import FlattenObservation
from sklearn.model_selection import train_test_split
import time

import tensorflow as tf

from stable_baselines3.dqn import MlpPolicy, MultiInputPolicy
from stable_baselines3.common.vec_env import DummyVecEnv
from stable_baselines3 import PPO
from stable_baselines3.common import monitor 
from stable_baselines3.common import logger

### Separando os Conjuntos Teste x Treino

In [3]:
# Importando os dados do Excel
df_dataset = pd.read_excel('./DadosGolfitSoccer_2024_07_03.xlsx')

# Removendo colunas desnecessárias
df_dataset.drop(columns=['Codigo', 'Ano_Avaliacao', 'Nome', 'CategoriaEtaria', 'Ano_Nascimento','CLUBE', 'PosicaoJogo', 'YOYO_Percurso'], inplace=True)

# Removendo as linhas com valores vazios
df_dataset.dropna(
    axis=0,
    how='any',
    subset=None,
    inplace=True
)

# Discretizando as colunas para diminuir o espaço de observações
df_dataset[df_dataset.columns]= df_dataset.round().astype('Int64')

In [4]:
# Extraindo os valores máximos e mínimos para cada coluna do DataFrame
# Cria dicionários para armazenar os valores máximos e mínimos para cada coluna
spaces_dict_max = {}
spaces_dict_min = {}

# Percorre as colunas do dataset
for column in df_dataset:
    # Extrai o valor máximo da coluna e armazena no dicionário
    spaces_dict_max[column] = df_dataset[column].max(axis=0)
    # Extrai o valor mínimo da coluna e armazena no dicionário
    spaces_dict_min[column] = df_dataset[column].min(axis=0)

    print(column,  spaces_dict_max[column],spaces_dict_min[column])

IdadeDecimal 17 13
Sucesso 1 0
Quartil_Nascimento 4 1
Dominância 3 1
Ambidestro 1 0
IdadeInicio 14 3
NivelDISPUTA 4 0
NivelVIToRIA 4 0
AtletanaFamilia 1 0
TempoPratica_anos 14 2
CompetenciaPercebida 10 5
EnvolvFamilia 38 13
LidarAdversidades 3 1
DesempenhoPressao 3 0
MetasPreparoMental 3 0
Concentracao 3 1
LivrePreocupacao 3 0
ConfiancaMotivacao 3 1
Treinabilidade 3 1
Coping 19 7
Competitivo 5 3
Vencedor 5 3
Determinado 5 3
PD 6 3
CSO 6 3
CSAB 6 2
ASM 6 2
SomaTACSIS 6 2
MassaCorporal 88 33
Estatura 188 143
AlturaSentado 98 46
GorduraPerc 37 6
ForcaPreensao 52 10
Flexibilidade 48 2
CMJ 44 20
Agilidade 9 7
Drible 15 8
DeltaDrible 7 0
Vel10m 2 1
Vel20m 4 3
AvaliacaoTreinador 5 1
Intangiveis 50 16


In [7]:
from gym.spaces import Dict, Discrete

observation_space = Dict({"position": Box(-1, 1, shape=(42,)), "color": Discrete(3)})

observation_space.sample()

OrderedDict([('color', 0),
             ('position',
              array([-0.34807643,  0.46198255,  0.13872552,  0.9810277 , -0.11435691,
                      0.4165001 ,  0.8388615 ,  0.18480267, -0.36041328, -0.0733355 ,
                      0.99273956,  0.67899096, -0.09793847, -0.71916676, -0.18295904,
                     -0.8668422 , -0.4070612 , -0.03655403,  0.5086717 ,  0.59678775,
                      0.16015223, -0.9279826 , -0.28801996, -0.17221628,  0.84917706,
                      0.5509966 , -0.75311184, -0.08284833, -0.5373875 ,  0.1520185 ,
                      0.10294146,  0.59690714, -0.49336416, -0.3579151 ,  0.824263  ,
                     -0.8048916 , -0.29614186, -0.04349132,  0.777123  ,  0.419413  ,
                      0.9173894 ,  0.50964177], dtype=float32))])

In [9]:
# Separando os conjuntos de teste e treino
df_train, df_test = train_test_split(df_dataset, test_size=0.2)

# Retirando a coluna da classe - Sucesso
df_train_X = np.array(df_train.drop('Sucesso', axis=1))
# Convertendo a coluna para o tipo que será usado pelo modelo
df_train_y = np.array(df_train['Sucesso'])

# Testar o uso das dimensões expandidas
# df_train_X = np.expand_dims(df_train_X, 1)
# df_train_y = np.expand_dims(df_train_y, 1)

In [71]:
#### TODO: automatizar esse processo 
# Testando um formato de observação no espaço
observation_space = Dict({
            "IdadeDecimal": Discrete(4, start=13),
            "Sucesso": Discrete(2),
            "Quartil_Nascimento": Discrete(4, start=1),
            "Dominancia": Discrete(3, start=1),
            "Ambidestro": Discrete(2),
            "IdadeInicio": Discrete(11, start=3),
            "NivelDISPUTA": Discrete(5),
            "NivelVIToRIA": Discrete(5),
            "AtletanaFamilia": Discrete(2),
            "TempoPratica_anos": Discrete(12, start=2),
            "CompetenciaPercebida": Discrete(5, start=5),
            "EnvolvFamilia": Discrete(25, start=13),
            "LidarAdversidades": Discrete(3, start=1),
            "DesempenhoPressao": Discrete(4),
            "MetasPreparoMental": Discrete(4),
            "Concentracao": Discrete(3, start=1),
            "LivrePreocupacao": Discrete(4),
            "ConfiancaMotivacao": Discrete(3, start=1),
            "Treinabilidade": Discrete(3, start=1),
            "Coping": Discrete(12, start=7),
            "Competitivo": Discrete(3, start=3),
            "Vencedor": Discrete(3, start=3),
            "Determinado": Discrete(3, start=3),
            "PD": Discrete(4, start=3),
            "CSO": Discrete(4, start=3),
            "CSAB": Discrete(5, start=2),
            "ASM": Discrete(5, start=2),
            "SomaTACSIS": Discrete(5, start=2),
            "MassaCorporal": Discrete(56, start=33),
            "Estatura": Discrete(46, start=143),
            "AlturaSentado": Discrete(53, start=46),
            "GorduraPerc": Discrete(32, start=6),
            "ForcaPreensao": Discrete(43, start=10),
            "Flexibilidade": Discrete(47, start=2),
            "CMJ": Discrete(25, start=20),
            "Agilidade": Discrete(3, start=7),
            "Drible": Discrete(8, start=8),
            "DeltaDrible": Discrete(8),
            "Vel10m": Discrete(2, start=1),
            "Vel20m": Discrete(2, start=3),
            "AvaliacaoTreinador": Discrete(5, start=1),
            "Intangiveis": Discrete(35, start=16)
        },seed=42)

# Gerando uma amostra de observação
observation_space.sample()

OrderedDict([('ASM', 2),
             ('Agilidade', 9),
             ('AlturaSentado', 52),
             ('Ambidestro', 1),
             ('AtletanaFamilia', 0),
             ('AvaliacaoTreinador', 4),
             ('CMJ', 38),
             ('CSAB', 4),
             ('CSO', 3),
             ('CompetenciaPercebida', 9),
             ('Competitivo', 3),
             ('Concentracao', 3),
             ('ConfiancaMotivacao', 2),
             ('Coping', 15),
             ('DeltaDrible', 4),
             ('DesempenhoPressao', 0),
             ('Determinado', 5),
             ('Dominancia', 2),
             ('Drible', 15),
             ('EnvolvFamilia', 17),
             ('Estatura', 153),
             ('Flexibilidade', 15),
             ('ForcaPreensao', 49),
             ('GorduraPerc', 14),
             ('IdadeDecimal', 16),
             ('IdadeInicio', 3),
             ('Intangiveis', 18),
             ('LidarAdversidades', 2),
             ('LivrePreocupacao', 1),
             ('MassaCorpo

### Criando o ambiente para classificação

In [25]:
# Env: uma classe python de alto-nível que representa um MDP(Processo de Decisão de Markov)
#      permite ao usuário gerar o estado inicial, transição de estados e visualização do ambiente
# Instanciando um ambiente a partir do ambiente fornecido pelo Gym
class PlayersEnv(Env):
    def __init__(self, majority_class=0, minority_class=1, row_per_episode=1, dataset=(df_train_X, df_train_y)):
        # As ações que o agente podem tomar são discretas, envolvendo classificar entre 2 classes
        self.action_space = Discrete(2)
        
        # O espaço de observações consiste nos jogadores que o agente deve classificar - 42 features para cada jogador
        # obersevation = np.array([[np.finfo('float32').max] * 42], dtype=np.float32)
        self.observation_space = Box(0, 60, shape=(41,))
        # }, seed=42)
        # self.observation_space = Box(-obersevation, obersevation, shape=(1, 42), dtype=np.float32)

        # Definindo qual as classe majoritária/minoritária do dataset - Para lidar com desbalanceamento dos dados
        self.majority_class = majority_class
        self.minority_class = minority_class

        # self.observations_space = Box(low=0, high=1, shape=())
        # Estado inicial em que o agente começa a interação
        self.state = 0
        # Tamanho do episódio == quantidade de instâncias (jogadores) a serem classificados
        self.episode_length = 0

        # Definição de parâmetros
        self.row_per_episode = row_per_episode
        self.step_count = 0
        self.x, self.y = dataset
        self.dataset_idx = 0
        

    # Função que especifica o que o agente fará a cada passo - Representa uma ação do agente
    # Recebe a ação tomada pelo agente como parâmetro
    def step(self, action):
        # Verificar se o agente classificou certo
        # Diferenciar recompensas para a classe maioritária e minoritária
        # Utilizar a fórmula definida pelos autores na metodologia do artigo

        # Flag que define se a tomada de decisão está finalizada
        done = False

        # Definição das recompensas caso o agente tome a decisão CORRETA
        if action == self.expected_action:
            print("Jogador classificado corretamente")
            match self.expected_action:
                # Caso o atleta seja da classe majoritária recebe uma recompensa menor
                case self.majority_class:
                    print("Classe: 0")
                    print("Recompensa: 1")
                    reward = 1
                # Caso o atleta seja da classe minoritária recebe uma recompensa maior
                case self.minority_class:
                    print("Classe: 1")
                    print("Recompensa: 100")
                    reward = 100

        # Definição das recompensas caso o agente tome a decisão INCORRETA
        if action != self.expected_action:
            print("Jogador classificado incorretamente")
            match self.expected_action:
                # Caso o atleta seja da classe majoritária recebe uma recompensa menor
                case self.majority_class:
                    print("Classe: 0")
                    print("Recompensa: -1")
                    reward = -1
                # Caso o atleta seja da classe minoritária recebe uma recompensa maior
                case self.minority_class:
                    print("Classe: 0")
                    print("Recompensa: -100")
                    reward = -100

        # Passa para a próxima observação do dataset
        observation = self._next_observation()

        # Aumenta a contagem de decisões tomadas
        self.step_count += 1

        # Caso a quantidade de passos definidos por episódio tenha sido alcançada
        if self.step_count >= self.row_per_episode:
            done = True

        return observation, reward, done, {}  

    # Função que reseta o ambiente para treinar novamente
    def reset(self):
        # Zera a contagem de passos
        self.step_count = 0
        # Define a nova observação inicial
        observation = self._next_observation()
        # Retorna a observação inicial
        return observation

    # Subrotina que retorna a próxima observação para o agente
    def _next_observation(self):
        # Define um índice de forma aleatória para a próxima observação
        next_observation_idx = random.randint(0, len(self.x) - 1)
        # Define a ação esperada para a observação - Retorna o rótulo da observação
        self.expected_action = int(self.y[next_observation_idx])
        # Extrai a observação com o índice aleatório definido anteriormente
        observation = self.x[next_observation_idx]

        return observation
        

In [26]:
from gym.envs.registration import register

register(
    id='PlayersEnv',
    entry_point='PlayersEnv',
    max_episode_steps=3000000,
)

### Treinamento do modelo

In [28]:
# Configurando a pasta para logs
# logger.configure('./logs', ['stdout', 'tensorboard'])
# Instanciando o ambiente para treinamento
environment = PlayersEnv()
# environment.observation_space.shape

# environment = FlattenObservation(environment)

environment.observation_space.shape
# Encapsulando o ambiente para logs
# environment = DummyVecEnv([lambda: monitor.Monitor(PlayersEnv(), logger.Logger('./logs',['stdout', 'tensorboard']).get_dir())])

# Inicializando o modelo PPO
model = PPO(
    'MlpPolicy', environment,
    verbose=1)

# Treinando o modelo
model.learn(total_timesteps=int(1.2e4))
# model.learn(total_timesteps=int(10))

# Medindo o tempo de treinamento
start_time = time.time()
print("Tempo de Treinamento PPO:", time.time() - start_time)


Using cpu device
Wrapping the env with a `Monitor` wrapper
Wrapping the env in a DummyVecEnv.
Jogador classificado incorretamente
Classe: 0
Recompensa: -100
Jogador classificado incorretamente
Classe: 0
Recompensa: -1
Jogador classificado incorretamente
Classe: 0
Recompensa: -100
Jogador classificado incorretamente
Classe: 0
Recompensa: -1
Jogador classificado corretamente
Classe: 0
Recompensa: 1
Jogador classificado incorretamente
Classe: 0
Recompensa: -100
Jogador classificado incorretamente
Classe: 0
Recompensa: -1
Jogador classificado incorretamente
Classe: 0
Recompensa: -100
Jogador classificado corretamente
Classe: 0
Recompensa: 1
Jogador classificado corretamente
Classe: 0
Recompensa: 1
Jogador classificado incorretamente
Classe: 0
Recompensa: -1
Jogador classificado incorretamente
Classe: 0
Recompensa: -1
Jogador classificado corretamente
Classe: 0
Recompensa: 1
Jogador classificado incorretamente
Classe: 0
Recompensa: -1
Jogador classificado incorretamente
Classe: 0
Recompensa